<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imputando-con-kNN-(kNN-imputation)" data-toc-modified-id="Imputando-con-kNN-(kNN-imputation)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imputando con kNN (kNN imputation)</a></span><ul class="toc-item"><li><span><a href="#Fundamentos-de-la-Imputación-con-kNN-los-missing-values-de-variables-numéricas." data-toc-modified-id="Fundamentos-de-la-Imputación-con-kNN-los-missing-values-de-variables-numéricas.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Fundamentos de la Imputación con kNN los missing values de variables numéricas.</a></span></li><li><span><a href="#Imputamos-en-el-Train" data-toc-modified-id="Imputamos-en-el-Train-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Imputamos en el Train</a></span></li><li><span><a href="#Imputamos-en-el-Test" data-toc-modified-id="Imputamos-en-el-Test-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Imputamos en el Test</a></span></li><li><span><a href="#OBSERVACIONES-IMPORTANTES:" data-toc-modified-id="OBSERVACIONES-IMPORTANTES:-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>OBSERVACIONES IMPORTANTES:</a></span><ul class="toc-item"><li><span><a href="#Pregunta:" data-toc-modified-id="Pregunta:-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Pregunta:</a></span></li><li><span><a href="#Respuesta:" data-toc-modified-id="Respuesta:-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Respuesta:</a></span></li><li><span><a href="#Pregunta" data-toc-modified-id="Pregunta-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>Pregunta</a></span></li><li><span><a href="#Respuesta" data-toc-modified-id="Respuesta-1.4.4"><span class="toc-item-num">1.4.4&nbsp;&nbsp;</span>Respuesta</a></span></li></ul></li></ul></li></ul></div>

![IES21](img/logo_ies.png)

# Situación Profesional 4: Preparación de los Datos
# Limpieza de los Datos: kNN Imputation para variables numéricas

## Imputando con kNN (kNN imputation)

### Fundamentos de la Imputación con kNN los missing values de variables numéricas.

En nuestro interés de mantener la mayor cantidad de observaciones posibles del DataSet original hemos aprendido a utilizar los métodos más *simples* (imputando con la media, mediana, moda de cada feature o un valor constante), sin embargo también es posible utilizar métodos más sofisticados. 

Uno de los que mejores resultados da es nuestro conocido kNN. Recuerda que habíamos comentado que kNN aparecerá en varias tareas en el flujo de machine learning? Veamos cómo funciona y cómo se implementa en Scikit-Learn

Supongamos que tenemos el siguiente DataFrame con las variables X, ya listo para imputar ( en un problema de Machine Learning ya hubiéramos *retirado* el Test Set de circulación y estaríamos trabajando sólo en el Train Set ... la aclaración vale porque si sólo estuviéramos haciendo análisis estadístico de los datos no sería necesario separar el Test Set  ;) )

In [1]:
import numpy as np
import pandas as pd

In [2]:
X_Train=pd.DataFrame([[1,3, np.nan],[2,3,5],[14,10,9],[1,2,3],[np.nan,12,8],[10,13,7],[2,4,2],[13,10,8]])
X_Train.columns=['x1','x2','x3']
X_Train

,x1,x2,x3
0,1.0,3,NaN
1,2.0,3,5.0
2,14.0,10,9.0
3,1.0,2,3.0
4,NaN,12,8.0
5,10.0,13,7.0
6,2.0,4,2.0
7,13.0,10,8.0


La imputación con kNN hará algo que ya hemos hecho varias veces en ejercicios anteriores ... cuando trabajábamos con las películas por ejemplo ...  

Veamos la primer observación, ella tiene un nan correspondiente a la variable o feature x<sub>3</sub>, nuestro plan sería:  
- buscar las k observaciones más cercanas (en este caso podríamos elegir k=2 porque tenemos pocas observaciones, pero ya sabemos que este es el punto débil de kNN).
- promediar el valor de x<sub>3</sub> de estas k observaciones más cercanas y asignar este resultado al NaN en la primer observación. Por supuesto, el promedio podría ser ponderado -según la distancia- o no. Generalmente se utlizan opciones simples en esta etapa, pero ante una situación desesperante se podría probar.

Éso es justamente lo que nos permitirá hacer sklearn con:

~~~
sklearn.impute.KNNImputer()
~~~

La documentación completa se encuentra en : https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer 

y la sintaxis es:  

~~~
sklearn.impute.KNNImputer(*, missing_values=nan, n_neighbors=5, weights='uniform', metric='nan_euclidean', copy=True, add_indicator=False)
~~~

Las opciones más importantes son una mezcla de las que ya hemos visto para kNN y para SimpleImputer:  

- missing_values: igual que en SimpleImputer,  podemos dejar el valor por defecto, nan, o indicar otro valor al que querramos imputar
- n_neighbors, weights, y metric, igual que en kNN.  

Vamos a aplicarlo a nuestro dataframe ... se utiliza de la misma manera que SimpleImputer ...

In [3]:
from sklearn.impute import KNNImputer

**1- Creamos** el imputador con las opciones preferidas, en nuestro caso 2 vecinos, los demás valores los dejamos por defecto:

In [4]:
imputador_knn=KNNImputer(n_neighbors=2)

**2- Entrenamos** el imputador con el método **fit**, usando los datos del DataFrame:

In [5]:
imputador_knn.fit(X_Train)

KNNImputer(n_neighbors=2)

**3- Aplicamos** con **transform** el imputador al conjunto de datos que necesitemos, seguramente al mismo Train , pero en un flujo de Machine Learning también deberíamos hacerlo sobre el Test y en producción sobre cualquier nueva observación que nos llegara para pronosticar.

### Imputamos en el Train

In [6]:
Train_Transformado=imputador_knn.transform(X_Train)
# Train_Transformado 

# Si lo queremos como DataDrame:
X_Train_Transformado=pd.DataFrame(Train_Transformado,columns=X_Train.columns)
X_Train_Transformado

,x1,x2,x3
0,1.0,3.0,4.0
1,2.0,3.0,5.0
2,14.0,10.0,9.0
3,1.0,2.0,3.0
4,11.5,12.0,8.0
5,10.0,13.0,7.0
6,2.0,4.0,2.0
7,13.0,10.0,8.0


Podemos verificar "a ojo" lo que hizo para estar seguros de cómo procedió.   
De dónde salió el valor 4 para el valor de x<sub>3</sub> de la primer observación (índice 0)?  

Inspeccionando en forma visual, podemos ver que las 2 observaciones más cercanas a la primera son:  

- la segunda (índice 1) que tiene un 5 en la columna de x<sub>3</sub>
- la cuarta (índice 3) que tiene un 3 en la columna de x<sub>3</sub>

por ende calculó el promedio simple entre estos dos valores, que es 4 y lo asignó.   

De la misma manera puede verificar cómo asignó el 11.5 al elemento de la columna x<sub>1</sub> de la quinta observación (índice 4).  


### Imputamos en el Test

Si tuviéramos un X_Test al cual aplicarlo, simplemente le aplicaríamos el método **transform**, de la siguiente manera:

In [7]:
# supongamos que el siguiente sea el X_Test
X_Test=pd.DataFrame([[4.5,2,np.nan],[3,3,3],[10,np.nan,12]])
X_Test.columns=X_Train.columns
X_Test

,x1,x2,x3
0,4.5,2.0,NaN
1,3.0,3.0,3.0
2,10.0,NaN,12.0


In [8]:
# efectuamos la imputación
Test_Transformado=imputador_knn.transform(X_Test)

# Lo pasamos a un DataFrame por si lo necesitáramos en ese formato.
X_Test_Transformado=pd.DataFrame(Test_Transformado, columns=X_Test.columns)
X_Test_Transformado

,x1,x2,x3
0,4.5,2.0,3.5
1,3.0,3.0,3.0
2,10.0,11.5,12.0


### OBSERVACIONES IMPORTANTES: 

> Fíjese que por ser kNN un método que *no aprende* sino que se vuelven a utilizar todas las observaciones en cada imputación, por defecto imputó al valor de x<sub>2</sub> faltante en la última observación del Test Set o en la nueva_observación, aunque en el Train Set no habíamos tenido que efectuar ningún pronóstico en esa columna.

> Por otro lado, como kNN *no aprende*, no nos olvidemos que **es lento**, y que aún en producción ingresando de a una nueva observación por vez, como podría ser el caso de un problema online, deberá reutilizar todo el Train Set para imputar los missing values!

#### Pregunta:  

En una aplicación que debe pronosticar nuevas observaciones que ingresan en producción y en tiempo real, qué imputación para los missing values  propondría realizar?


#### Respuesta:  

La media, la mediana, o un valor constante si es un problema de Regresión y la moda o un valor constante en el caso de Clasificación, calculados sobre todo el dataset disponible. Estos valores estadísticos se calculan una única vez sobre el X, y se utilizan muy velozmente, en cambio kNN debería efectuar los cálculos de distancia sobre todas las observaciones del X para cada nueva observación que ingresara.

En Scikit-Learn:

> El imputador kNNImputer puede usarse de la misma manera que el SimpleImputer y utilizarlo dentro de un **ColumnTransformer**.

#### Pregunta

Supongamos que queremos imputar un NaN en una columna donde ya hay otros NaN correspondientes a otras observaciones, funciona igual kNN?  





#### Respuesta  

Si.  

Al principio comentábamos que no se podía utilizar *cualquier* modelo para imputar missing values porque tenía que ser un método *inmune* a los missing values, ahora quiero aclararle que nos referíamos a la posible presencia de varios missing values en las mismas columnas donde deseamos pronosticar otros missing values, es decir que el método también funcionaría bien en dataframes como el siguiente, donde tenemos varios np.nan en cada columna!


In [9]:
X_Train2=pd.DataFrame([[1,3, np.nan],[2,3,5],[14,10,9],[1,np.nan,3],[np.nan,12,8],[10,13,np.nan],[np.nan,4,2],[13,10,8]])
X_Train2.columns=['x1','x2','x3']
X_Train2

,x1,x2,x3
0,1.0,3.0,NaN
1,2.0,3.0,5.0
2,14.0,10.0,9.0
3,1.0,NaN,3.0
4,NaN,12.0,8.0
5,10.0,13.0,NaN
6,NaN,4.0,2.0
7,13.0,10.0,8.0
